In [ ]:
path = '../data/prueba/pequena.tif'

def get_bands_positions(path):
    """
    Permite obtener las posiciones y los nombres de las bandas
    a partir de los metadatos de la imágen tif.
    
    [(1, 'green'), (2, 'swir2'), (3, 'nir'), (4, 'blue'), (5, 'red'), (6, 'swir1')]
    """
    ds = gdal.Open(path)
    # Extracción de los metadatos del tif
    metadata = ds.GetMetadata()
    # Extracción de la posición y nombre de las bandas
    positions = [ (int(k[-1]) - 1,v) for k,v in metadata.items() if 'Band_' in k ]
    
    return positions


def load_ideam_geotif(path):
    
    positions = get_bands_positions(path)
    
    data_array = xr.open_rasterio(path)

    # Define las coordenadas de los puntos
    coords = { 
        'x':data_array.x.values, 
        'y':data_array.y.values 
    }
    
    # Define el orden de las dimensiones
    dims = ('y','x')
    
    # Define los metadatos o atributos
    attrs = {
        'crs': data_array.crs,
        'transform': data_array.transform
    }
    
    data = {}
    for band_index, band_name in positions:
        data[band_name] = xr.DataArray(
            data=data_array.values[band_index], 
            coords=coords, 
            dims=dims, 
            name=band_name, 
            attrs=attrs
        )
    
    return data

tif = load_ideam_geotif(path)
tif
